# 1. Colab 환경을 설정하기

In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2

     |████████████████████████████████| 344 kB 6.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=bf154e6607a76c506907b132215524b70a468e5cff28dd65d75e5aabd556f5a4
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 18.8 MB/s 
     |████████████████████████████████| 2.5 MB 32.2 MB/s 
     |████████████████████████████████| 3.3 MB 51.3 MB/s 
     |████████████████████████████████| 895 kB 66.8 MB/s 


In [2]:
!pip install torch

# 2. github에서 KoBERT 파일을 로드 및 KoBERT모델 불러오기

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-edodfk_j/kobert-tokenizer_ecc982adafc640b9b52b0274d4042efc
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-edodfk_j/kobert-tokenizer_ecc982adafc640b9b52b0274d4042efc
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=9806edec97a08ee23c4da633a0b5604e41abe9f8d64fb833f2d94a5327f63967
  Stored in directory: /tmp/pip-ephem-wheel-cache-evfoo7ue/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-gcgn5u9a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-gcgn5u9a
     |████████████████████████████████| 132 kB 32.2 MB/s 
     |████████████████████████████████| 4.5 MB 67.5 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 8.6 MB 65.2 MB/s 
     |████████████████████████████████| 127 kB 71.1 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=7346a9d5cd052574b91bcfe6e35ff12d27ae5e1139ae75bb620c3e175bbf7ebd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ft2n5009/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does no

In [5]:
from kobert.utils.utils import download, get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.mxnet_kobert import get_mxnet_kobert_model
from kobert.onnx_kobert import get_onnx_kobert_model

__all__ = ("download", "get_tokenizer", "get_pytorch_kobert_model" ,"get_mxnet_kobert_model", "get_onnx_kobert_model")

In [6]:
import gluonnlp as nlp
import mxnet as mx
from gluonnlp.model import BERTEncoder, BERTModel

from kobert import download, get_tokenizer


def get_mxnet_kobert_model(
    use_pooler=True,
    use_decoder=True,
    use_classifier=True,
    ctx=mx.cpu(0),
    cachedir=".cache",
):
    def get_kobert_model(
        model_file,
        vocab_file,
        use_pooler=True,
        use_decoder=True,
        use_classifier=True,
        ctx=mx.cpu(0),
    ):
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )

        predefined_args = {
            "attention_cell": "multi_head",
            "num_layers": 12,
            "units": 768,
            "hidden_size": 3072,
            "max_length": 512,
            "num_heads": 12,
            "scaled": True,
            "dropout": 0.1,
            "use_residual": True,
            "embed_size": 768,
            "embed_dropout": 0.1,
            "token_type_vocab_size": 2,
            "word_embed": None,
        }

        encoder = BERTEncoder(
            num_layers=predefined_args["num_layers"],
            units=predefined_args["units"],
            hidden_size=predefined_args["hidden_size"],
            max_length=predefined_args["max_length"],
            num_heads=predefined_args["num_heads"],
            dropout=predefined_args["dropout"],
            output_attention=False,
            output_all_encodings=False,
        )

        # BERT
        net = BERTModel(
            encoder,
            len(vocab_b_obj.idx_to_token),
            token_type_vocab_size=predefined_args["token_type_vocab_size"],
            units=predefined_args["units"],
            embed_size=predefined_args["embed_size"],
            word_embed=predefined_args["word_embed"],
            use_pooler=use_pooler,
            use_decoder=use_decoder,
            use_classifier=use_classifier,
        )
        net.initialize(ctx=ctx)
        net.load_parameters(model_file, ctx, ignore_extra=True)
        return (net, vocab_b_obj)

    mxnet_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/mxnet_kobert_45b6957552.params",
        "chksum": "45b6957552",
    }

    # download model
    model_info = mxnet_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(
        model_path, vocab_path, use_pooler, use_decoder, use_classifier, ctx
    )


if __name__ == "__main__":
    import mxnet as mx
    from kobert import get_mxnet_kobert_model

    input_id = mx.nd.array([[31, 51, 99], [15, 5, 0]])
    input_mask = mx.nd.array([[1, 1, 1], [1, 1, 0]])
    token_type_ids = mx.nd.array([[0, 0, 1], [0, 1, 0]])
    model, vocab = get_mxnet_kobert_model(use_decoder=False, use_classifier=False)
    encoder_layer, pooled_output = model(input_id, token_type_ids)
    print(pooled_output.shape)
    print(vocab)
    print(encoder_layer[0])

/content/.cache/mxnet_kobert_45b6957552.params[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
(2, 768)
Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

[[-0.24610384  0.24282105  0.2589529  ... -0.48613426 -0.07305261
   0.07560529]
 [-0.2478311   0.24200444  0.2552045  ... -0.48771802 -0.07270504
   0.07536069]
 [-0.24721529  0.24196576  0.25606254 ... -0.48743096 -0.07327029
   0.07650222]]
<NDArray 3x768 @cpu(0)>


In [7]:
from kobert import download


def get_onnx_kobert_model(cachedir=".cache"):
    """Get KoBERT ONNX file path after downloading"""
    onnx_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert.onnx1.8.0.onnx",
        "chksum": "6f6610f2e3b61da6de8dbce",
    }

    model_info = onnx_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    return model_path


def make_dummy_input(max_seq_len):
    def do_pad(x, max_seq_len, pad_id):
        return [_x + [pad_id] * (max_seq_len - len(_x)) for _x in x]

    input_ids = do_pad([[31, 51, 99], [15, 5]], max_seq_len, pad_id=1)
    token_type_ids = do_pad([[0, 0, 0], [0, 0]], max_seq_len, pad_id=0)
    input_mask = do_pad([[1, 1, 1], [1, 1]], max_seq_len, pad_id=0)
    position_ids = list(range(max_seq_len))
    return (input_ids, token_type_ids, input_mask, position_ids)


if __name__ == "__main__":
    import onnxruntime
    import numpy as np
    from kobert import get_onnx_kobert_model

    onnx_path = get_onnx_kobert_model()
    dummy_input = make_dummy_input(max_seq_len=512)
    so = onnxruntime.SessionOptions()
    sess = onnxruntime.InferenceSession(onnx_path)
    outputs = sess.run(
        None,
        {
            "input_ids": np.array(dummy_input[0]),
            "token_type_ids": np.array(dummy_input[1]),
            "input_mask": np.array(dummy_input[2]),
            "position_ids": np.array(dummy_input[3]),
        },
    )
    print(outputs[-2][0])

/content/.cache/kobert.onnx1.8.0.onnx[██████████████████████████████████████████████████]
[[-0.2365442   0.241829    0.21331948 ... -0.51101243 -0.03597119
   0.08153652]
 [-0.23781252  0.24054067  0.21029475 ... -0.5120347  -0.03533047
   0.08124782]
 [-0.23773046  0.24072771  0.21037376 ... -0.5113285  -0.03561583
   0.08166732]
 ...
 [-0.23613384  0.24219985  0.21134701 ... -0.5129691  -0.03717504
   0.08178963]
 [-0.23613562  0.242193    0.21135321 ... -0.51296335 -0.03717914
   0.08178397]
 [-0.23613118  0.24219249  0.21135503 ... -0.512966   -0.03717308
   0.0817885 ]]


In [8]:
import os
from zipfile import ZipFile
import torch
from transformers import BertModel
import gluonnlp as nlp

from kobert import download, get_tokenizer


def get_pytorch_kobert_model(ctx="cpu", cachedir=".cache"):
    def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

    pytorch_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert_v1.zip",
        "chksum": "411b242919",  # 411b2429199bc04558576acdcac6d498
    }

    # download model
    model_info = pytorch_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    zipf = ZipFile(os.path.expanduser(model_path))
    zipf.extractall(path=cachedir_full)
    model_path = os.path.join(os.path.expanduser(cachedir), "kobert_from_pretrained")
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(model_path, vocab_path, ctx)


if __name__ == "__main__":
    import torch
    from kobert import get_pytorch_kobert_model

    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    model, vocab = get_pytorch_kobert_model()
    sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)
    print(pooled_output.shape)
    print(vocab)
    print(sequence_output[0])

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
torch.Size([2, 768])
Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward0>)


In [9]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

In [10]:
if __name__ == "__main__":
    import torch
    from kobert import get_pytorch_kobert_model

    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    model, vocab = get_pytorch_kobert_model()
    sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)
    print(pooled_output.shape)
    print(vocab)
    print(sequence_output[0])

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
torch.Size([2, 768])
Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward0>)


# 3. 필요한 라이브러리 불러오기

In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# 4. 데이터셋 불러오기

In [13]:
data1 = pd.read_csv('4차년도.csv', encoding='cp949')
data2 = pd.read_csv('5차년도.csv', encoding='cp949')
data3 = pd.read_csv('5차년도_2차.csv', encoding='cp949')

In [14]:
raw_df = pd.concat([data1, data2, data3])

In [15]:
raw_df["상황"].value_counts()

sad          11152
anger         8372
disgust       4098
fear          3818
happiness     3648
angry         2659
neutral       2374
sadness       2226
surprise      1468
Name: 상황, dtype: int64

In [16]:
df = raw_df.loc[(raw_df["상황"] == "angry") | (raw_df["상황"] == "anger") | (raw_df["상황"] == "sadness") | (raw_df["상황"] == "sad") | (raw_df["상황"] == "happiness") | (raw_df["상황"] == "neutral")]

In [17]:
df["상황"].value_counts()

sad          11152
anger         8372
happiness     3648
angry         2659
neutral       2374
sadness       2226
Name: 상황, dtype: int64

In [18]:
df.loc[((df['상황'] == "angry") | (df['상황'] == "anger")), '상황'] = 1  #분노 
df.loc[((df['상황'] == "sadness") | (df['상황'] == "sad")), '상황'] = 2  #슬픔
df.loc[(df['상황'] == "neutral"), '상황'] = 3  #중립 
df.loc[(df['상황'] == "happiness"), '상황'] = 4  #행복

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [19]:
data_list = []
for ques, label in zip(df['발화문'], df['상황'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)
data_list[:10]

[['어, 청소 니가 대신 해 줘!', '1'],
 ['둘 다 청소 하기 싫어. 귀찮아.', '1'],
 ['둘 다 하기 싫어서 화내.', '1'],
 ['그럼 방세는 어떡해.', '1'],
 ['권태긴줄 알았는데 다른 사람이 생겼나보더라고.', '2'],
 ['그냥 걷고 있어.', '2'],
 ['어. 고등학교 동창인데 이렇게 더럽게 쓸줄 몰랐어.', '1'],
 ['처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.', '2'],
 ['내가 애정 표현을 잘 못해서 자주 싸우긴 했어.', '2'],
 ['오늘 헤어졌어.', '2']]

# 5. 입력 데이터셋을 토큰화하기

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters 예시 코드 값과 동일하게 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=1234)

In [23]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [28]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
#BERT 모델 불러오기
model1 = BERTClassifier(model,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model1.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model1.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [31]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model1.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model1.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/381 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.020601272583008 train acc 0.125
epoch 1 batch id 201 loss 0.38852760195732117 train acc 0.6203358208955224
epoch 1 train acc 0.7573955599300087


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9534272693452381


  0%|          | 0/381 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.005944606848061085 train acc 1.0
epoch 5 batch id 201 loss 0.02084733359515667 train acc 0.986396144278607
epoch 5 train acc 0.9881479658792651


  0%|          | 0/96 [00:00<?, ?it/s]

epoch 5 test acc 0.9537527901785715


In [33]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()


            if np.argmax(logits) == 1:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("중립이")
            elif np.argmax(logits) == 4:
                test_eval.append("행복이")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 피곤하다


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 포항 못 간다.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 바보
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 밥이 너무 맛있다
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 부산간다.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 행복하다
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 좋아
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 맛있다
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 멍청해
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 화난다
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 아프다
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 헤어졌어
>> 입력하신 내용에서 슬픔이 느껴집니다.


